In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import Butterfly_identification.preprocessbutterfly as preproc
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.metrics import CategoricalAccuracy

from tensorflow.keras.applications.vgg16 import preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from os.path import join, sep
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow.strings import split
from tensorflow.io import decode_jpeg, read_file
from tensorflow import argmax
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
import gc
from tensorflow.compat.v1 import ConfigProto , Session, InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session

In [4]:
from tensorflow.data.experimental import cardinality

In [5]:
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(Session(config=config));
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2021-08-30 16:13:16.475753: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-30 16:13:16.541860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 16:13:16.551857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 16:13:16.552355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# Get Data

In [6]:
df_train,df_val,df_test = preproc.get_data()

In [7]:
df_train.shape

(36536, 15)

In [8]:
df_train = preproc.feature_engineering(df_train)
df_val = preproc.feature_engineering(df_val)
df_test = preproc.feature_engineering(df_test)

## Generator implementation

In [9]:
PATH_TRAIN = '/home/zac/code/Em3line/Butterfly_identification/raw_data/IGM_labels/Train/'
PATH_VAL = '/home/zac/code/Em3line/Butterfly_identification/raw_data/IGM_labels/Val/'
PATH_TEST = '/home/zac/code/Em3line/Butterfly_identification/raw_data/IGM_labels/Test/'

In [10]:
df_train = preproc.get_data_minphoto(df_train)
df_train = preproc.resampling(df_train)

In [11]:
df_val = preproc.filter_val_test(df_train, df_val)
df_test = preproc.filter_val_test(df_train, df_test)

In [12]:
df_train['full_path'] = df_train['species']+'/'+df_train['image_name']
df_val['full_path'] = df_val['species']+'/'+df_val['image_name']
df_test['full_path'] = df_test['species']+'/'+df_test['image_name']

In [13]:
black_list = list(PATH_TRAIN + df_train['full_path'])
black_list_val = list(PATH_VAL + df_val['full_path'])
black_list_test = list(PATH_TEST + df_test['full_path'])

In [14]:
train_names = df_train['species'].drop_duplicates()

In [15]:
test_predict = df_train['full_path'][0]

In [16]:
del df_train
del df_val
del df_test

In [17]:
filtered_train = [join(PATH_TRAIN, species) for species in black_list]
filtered_val = [join(PATH_VAL, species) for species in black_list_val]
filtered_test = [join(PATH_TEST, species) for species in black_list_test]

In [18]:
train_ds = Dataset.from_tensor_slices(filtered_train)
val_ds = Dataset.from_tensor_slices(filtered_val)
test_ds = Dataset.from_tensor_slices(filtered_test)

2021-08-30 16:13:28.068887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 16:13:28.069292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 16:13:28.069610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 16:13:28.070129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 16:13:28.070444: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [19]:
del filtered_train
del filtered_test
del filtered_val

In [20]:
def get_label(file_path,class_names=train_names):
  # convert the path to a list of path components
  parts = split(file_path, sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return argmax(one_hot)

In [21]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = decode_jpeg(img, channels=3)
    img = preprocess_input(img)
    # resize the image to the desired size
    return img


In [22]:
def process_path(file_path, class_names=train_names):
    label = get_label(file_path, class_names)
    # load the raw data from the file as a string
    img = read_file(file_path)
    img = decode_img(img)
    return img, label

In [23]:
AUTOTUNE = AUTOTUNE

In [24]:
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [25]:
def configure_for_performance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size=32)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

In [26]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

# Data Augmentation

In [27]:
data_augmentation = Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [28]:
IMG_SIZE = 128

resize_and_rescale = Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])


In [29]:
train_ds

<PrefetchDataset shapes: ((None, None, None, 3), (None,)), types: (tf.uint8, tf.int64)>

#augmentation = ImageDataGenerator(rotation_range=25, width_shift_range=0.2,
                  height_shift_range=0.2,  
                  zoom_range=0.1, horizontal_flip=True)

In [30]:
batch_size = 32
def prepare(ds, shuffle=False, augment=False):
    # Resize and rescale all datasets
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)


In [31]:
#train_ds = prepare(train_ds, shuffle=False, augment=True)

image_batch, label_batch = next(iter(train_ds))
print(len(label_batch))
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    print(image_batch.shape)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    label = label_batch[i]
    plt.title(train_names[label])
    plt.axis("off")

In [32]:
#train_ds = train_ds.map(augmentation.flow, num_parallel_calls=AUTOTUNE)

In [33]:
y_train = list(map(lambda x: x[1].numpy(), train_ds))

2021-08-30 16:13:29.832792: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [34]:
pd.DataFrame(y_train).max()

0     227.0
1     227.0
2     227.0
3     227.0
4     226.0
5     227.0
6     227.0
7     227.0
8     227.0
9     227.0
10    227.0
11    226.0
12    227.0
13    227.0
14    227.0
15    227.0
dtype: float64

In [35]:
y_train

[array([ 7, 13, 10, 13,  6,  7,  8, 18, 17,  4,  6, 17,  2, 13, 15,  8]),
 array([13,  2,  3,  2,  5,  1,  5,  3, 11,  7,  0, 11, 19,  4,  8,  1]),
 array([14,  4,  2, 17, 14,  0, 12, 11, 12,  2,  4, 17,  7, 19, 16,  5]),
 array([ 0, 17,  4,  1,  2, 19, 14, 15,  8,  0, 13, 15, 19, 16, 17,  6]),
 array([ 4,  0,  2, 13, 14,  9,  7,  0, 16,  8,  3,  7,  5,  9,  1, 13]),
 array([ 8, 12, 19, 11,  4,  2, 14,  2, 20,  3,  5,  3, 13, 21,  9,  2]),
 array([17,  6, 18, 14, 13, 17, 19, 11,  5, 17, 14,  6, 13, 12, 16, 15]),
 array([ 0,  7, 15, 16,  8, 17, 15,  1,  5,  0, 13, 12,  3, 18, 15, 17]),
 array([12, 10, 16,  1,  9,  6,  9,  0, 12, 14, 13, 18, 12,  1, 21,  6]),
 array([13, 20, 23, 15, 15, 23, 17,  1,  2, 10,  1, 11, 19, 22,  1, 12]),
 array([16,  7, 16,  7,  0,  2,  3,  0, 20, 20, 10,  7, 11,  1, 17, 21]),
 array([ 4,  3, 10,  8,  1,  2,  5, 23, 21, 16,  5, 22, 18,  6, 16,  4]),
 array([22, 21, 18,  2, 10,  1, 23, 25, 18,  8, 12, 24, 20,  5, 24, 17]),
 array([25, 20, 19,  5, 16, 16,  0, 10

# Model

In [36]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers, layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [69]:
def model_VGG16(train_ds, val_ds, test_ds, image_size=(128, 128, 3),  patience=10, learning_rate=0.001, nb_epochs=15, nb_couches_dense_layer=130):
    
    model = VGG16(weights="imagenet", include_top=False, input_shape = image_size)
    
    



    model.trainable = False
    
    #add last layers
    
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(nb_couches_dense_layer, activation='relu')
#     dropout_layer1 = layers.Dropout(.2, input_shape=(2,))
     dense_layer2 = layers.Dense(1000, activation='relu')
#     dropout_layer2 = layers.Dropout(.2, input_shape=(2,))
    prediction_layer = layers.Dense(228, activation='softmax')
    model_final = models.Sequential([
        data_augmentation,
        resize_and_rescale,
        model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    
    #build model
    opt = optimizers.Adam(learning_rate=learning_rate)
    model_final.compile(loss='sparse_categorical_crossentropy',
                  optimizer=opt,
                  metrics=CategoricalAccuracy())
    
    

    
    #set earlystopping
    es = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, restore_best_weights=True)
    
    #launch model
    history = model_final.fit(train_ds, 
                    validation_data=val_ds, 
                    epochs=nb_epochs,  
                    callbacks=[es])
    
    model_final.summary()
    
    #evaluate model
    res_vgg = model_final.evaluate(test_ds)
    
    test_accuracy_vgg = res_vgg[-1]
    
    return (f"test_accuracy_vgg = {round(test_accuracy_vgg,2)*100} %"), model_final

In [70]:
from Butterfly_identification.trainer import plot_history

In [ ]:
check, model = model_VGG16(train_ds, val_ds, test_ds, nb_epochs=50, nb_couches_dense_layer=2000)

Epoch 1/50
385/385 [==============================] - 73s 172ms/step - loss: 9.1390 - accuracy: 0.0968 - val_loss: 6.3594 - val_accuracy: 0.0271
Epoch 2/50
385/385 [==============================] - 50s 128ms/step - loss: 4.8636 - accuracy: 0.0994 - val_loss: 5.9025 - val_accuracy: 0.0182
Epoch 3/50
385/385 [==============================] - 67s 164ms/step - loss: 4.9479 - accuracy: 0.0864 - val_loss: 5.9283 - val_accuracy: 0.0187
Epoch 4/50
385/385 [==============================] - 62s 147ms/step - loss: 4.7584 - accuracy: 0.1025 - val_loss: 5.2743 - val_accuracy: 0.0235
Epoch 5/50
385/385 [==============================] - 54s 133ms/step - loss: 4.7043 - accuracy: 0.1014 - val_loss: 5.4249 - val_accuracy: 0.0253
Epoch 6/50
385/385 [==============================] - 50s 129ms/step - loss: 4.5703 - accuracy: 0.1113 - val_loss: 5.2561 - val_accuracy: 0.0221
Epoch 7/50
385/385 [==============================] - 51s 130ms/step - loss: 4.5121 - accuracy: 0.1254 - val_loss: 5.4818 - val_ac

In [53]:
from PIL import Image
#test_predict = Image.open(PATH_TRAIN+test_predict)
model.predict(preprocess_input(np.expand_dims(np.array(test_predict), axis=0)))
# PATH_TRAIN+test_predict

array([[1.]], dtype=float32)